<a href="https://colab.research.google.com/github/prateekmarda/prateekmarda/blob/main/onyx%20and%20tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TensorRT

In [1]:
!pip install onnxruntime-gpu onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.2 MB/s eta 0:00:00


In [2]:
!wget https://github.com/onnx/models/raw/refs/heads/main/validated/vision/classification/resnet/model/resnet50-v2-7.onnx -O resnet50.onnx


--2026-01-01 10:26:39--  https://github.com/onnx/models/raw/refs/heads/main/validated/vision/classification/resnet/model/resnet50-v2-7.onnx
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/onnx/models/refs/heads/main/validated/vision/classification/resnet/model/resnet50-v2-7.onnx [following]
--2026-01-01 10:26:39--  https://media.githubusercontent.com/media/onnx/models/refs/heads/main/validated/vision/classification/resnet/model/resnet50-v2-7.onnx
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102442452 (98M) [application/octet-stream]
Saving to: ‘resnet50.onnx’

resnet50.onnx   

In [3]:
import onnxruntime as ort
import numpy as np

In [4]:
# Load ONNX model with CUDA provider only
sess = ort.InferenceSession(
    "resnet50.onnx",
    providers=["CUDAExecutionProvider"]  # Remove TensorRT
)

print("Providers:", sess.get_providers())

# Prepare input data
input_name = sess.get_inputs()[0].name
input_data = np.random.random((1, 3, 224, 224)).astype(np.float32)

# Run inference
output = sess.run(None, {input_name: input_data})
print("Output shape:", output[0].shape)

Providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']
Output shape: (1, 1000)


In [5]:
import os
import onnx
import numpy as np
import onnxruntime as ort
import time
import torch
import torchvision.models as models

In [6]:
# 1. Load a pre-trained PyTorch model
print("Loading pre-trained ResNet-18 model...")
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.eval()

Loading pre-trained ResNet-18 model...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 131MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
# 2. Define model inputs and output file
model_file = "resnet18.onnx"
input_shape = (1, 3, 224, 224) # Standard ImageNet input size
dummy_input = torch.randn(*input_shape)

In [11]:
!pip install onnxscript
# 3. Export the model to ONNX format
print(f"Exporting PyTorch model to {model_file}...")
torch.onnx.export(
    model,
    dummy_input,
    model_file,
    export_params=True,
    opset_version=12,  # Compatible ONNX Opset Version
    do_constant_folding=True,
    input_names=["input_tensor"],
    output_names=["output_tensor"],
    dynamic_axes={'input_tensor': {0: 'batch_size'}} # Allow dynamic batch size
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 16.1 MB/s eta 0:00:00
Exporting PyTorch model to resnet18.onnx...


/tmp/ipython-input-1521737941.py:4: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0101 10:33:45.783000 800 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 12 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnx/version_converter.py", line 39, in convert_version
    converted_model_str = C.convert_version(model_str, target_version)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: /github/workspace/onnx/version_converter/adapters/axes_input_to_attribute.h:65: adapt: Asserti

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 40 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 18},
            producer_name='pytorch',
            producer_version='2.9.0+cu126',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input_tensor"<FLOAT,[s77,3,224,224]>
            ),
            outputs=(
                %"output_tensor"<FLOAT,[1,1000]>
            ),
            initializers=(
                %"fc.bias"<FLOAT,[1000]>{TorchTensor(...)},
                %"conv1.weight"<FLOAT,[64,3,7,7]>{Tensor(...)},
                %"layer1.0.conv1.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"layer1.0.conv2.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"layer1.1.conv1.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"layer1.1.conv2.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"layer2.0.conv1.weight"<FLOAT,[128,64,3,3]>{Tensor(...)},
            

In [12]:
# Verify and get tensor names
onnx_model = onnx.load(model_file)
onnx.checker.check_model(onnx_model)
input_name = onnx_model.graph.input[0].name
output_name = onnx_model.graph.output[0].name
print(f"✅ ONNX model created and checked successfully. Input name: {input_name}")

✅ ONNX model created and checked successfully. Input name: input_tensor


In [13]:
# Create dummy input data (a single batch of an image)
input_data = np.random.rand(*input_shape).astype(np.float32)
input_feed = {input_name: input_data}

# Define the providers, prioritizing TensorRT
providers = [
    ('TensorrtExecutionProvider', {
        'device_id': 0,
        'trt_fp16_enable': True, # Use faster FP16 precision (recommended on modern GPUs)
        'trt_max_workspace_size': 2147483648 # 2GB workspace limit
    }),
    'CUDAExecutionProvider' # Fallback to standard CUDA if TRT fails
]

print("\n--- Starting TensorRT Accelerated Session ---")
try:
    # Create the Inference Session
    sess_trt = ort.InferenceSession(model_file, providers = providers)

    # Check if the TensorRT provider was successfully loaded
    if 'TensorrtExecutionProvider' in sess_trt.get_providers():
        print("✅ Session successfully loaded with TensorRT EP.")
    else:
        # This is a safe check if TRT is available, but ORT sometimes abstracts this
        print("⚠️ Warning: TensorRT EP may not be the active provider. Check your GPU/CUDA versions.")

    # 🚀 Run the **First Inference (Warmup)** 🚀
    # This is the necessary, slow step where TensorRT compiles the optimized engine.
    print("Warming up (building TensorRT engine - this will be slow)...")
    start_time_build = time.time()
    _ = sess_trt.run([output_name], input_feed)
    build_time = time.time() - start_time_build
    print(f"TensorRT Engine build/warmup time: {build_time:.4f} seconds")

    # ⚡ Run Accelerated Inference ⚡
    # Subsequent runs use the compiled engine and demonstrate acceleration.
    num_runs = 50
    runtimes = []
    print(f"Running {num_runs} accelerated inferences for timing...")
    for _ in range(num_runs):
        start_time_inf = time.time()
        output_trt = sess_trt.run([output_name], input_feed)
        runtimes.append(time.time() - start_time_inf)

    avg_inf_time = np.mean(runtimes)
    print(f"\n--- TensorRT Inference Results ---")
    print(f"Average accelerated inference time: {avg_inf_time * 1000:.2f} ms")
    print(f"Output tensor shape: {output_trt[0].shape}")

except Exception as e:
    print(f"\n❌ Final error during TensorRT inference: {e}")
    print("Ensure all cells have been run in a GPU runtime.")


--- Starting TensorRT Accelerated Session ---
*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:560 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using [('TensorrtExecutionProvider', {'device_id': 0, 'trt_fp16_enable': True, 'trt_max_workspace_size': 2147483648}), 'CUDAExecutionProvider']
Falling back to ['CPUExecutionProvider'] and retrying.
****************************************
⚠️ Warning: TensorRT EP may not be the active provider. Check your GPU/CUDA versions.
Warming up (building TensorRT engine - this will be slow)...
TensorRT Engine build/warmup time: 0.0757 seconds
Running 50 accelerated inferences for timing...

--- TensorRT Inference Results ---
Average accelerated inference ti

# Optimization
### Sequence truncation

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "This is a very long sentence that will be truncated by the tokenizer."
max_length = 10

encoded = tokenizer(text, truncation=True, max_length=max_length)
print(encoded['input_ids'])

[101, 2023, 2003, 1037, 102]


In [6]:
# Static
encoded = tokenizer(["short", "a much longer sentence","A quick brown fox jumps over a lazy short dog"],
                    padding='max_length',
                    max_length=10)
print(encoded['input_ids'])

[[101, 2460, 102, 0, 0, 0, 0, 0, 0, 0], [101, 1037, 2172, 2936, 6251, 102, 0, 0, 0, 0], [101, 1037, 4248, 2829, 4419, 14523, 2058, 1037, 13971, 2460, 3899, 102]]


In [7]:
# Dynamic
encoded = tokenizer(["short", "a much longer sentence"],
                    padding='longest')
print(encoded['input_ids'])

[[101, 2460, 102, 0, 0, 0], [101, 1037, 2172, 2936, 6251, 102]]


In [8]:
from transformers import AutoTokenizer

# Example: T5 or XLM-R both use SentencePiece
tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokens = tokenizer.tokenize("This is an example sentence.")
print("SentencePiece Tokens:", tokens)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

SentencePiece Tokens: ['▁This', '▁is', '▁an', '▁example', '▁sentence', '.']


In [9]:
from tokenizers import ByteLevelBPETokenizer, BertWordPieceTokenizer

In [11]:
from tokenizers import BertWordPieceTokenizer

# Step 1: Define some example text (no file needed)
corpus = [
    "Tokenization is a core step in NLP.",
    "It splits text into smaller pieces called tokens.",
    "These tokens help models like BERT understand meaning."
]

# Step 2: Save to a temporary text file (required by tokenizer)
# Unfortunately, the current Tokenizers library still needs a file-like input for training.
# But we can create it dynamically inside Python without manually saving one.

with open("temp_corpus.txt", "w", encoding="utf-8") as f:
    for line in corpus:
        f.write(line + "\n")

# Step 3: Initialize and train tokenizer
tokenizer = BertWordPieceTokenizer(lowercase = True)
tokenizer.train(
    files = "temp_corpus.txt",
    vocab_size = 100,
    min_frequency = 1,
)

# Step 4: Use tokenizer directly (no reloading from vocab.txt)
encoded = tokenizer.encode("Tokenization divides text into tokens.")
print("Tokens:", encoded.tokens)


Tokens: ['tokeniz', '##ati', '##on', '[UNK]', 'te', '##xt', 'into', 'tokens', '.']


In [14]:
from tokenizers import ByteLevelBPETokenizer

# Step 1: Create a small example corpus
corpus = [
    "Tokenization is fun and powerful.",
    "Byte Pair Encoding splits words into subwords.",
    "This helps handle unknown tokens effectively."
]

# Step 2: Write corpus to a temporary file (required by the library)
with open("temp_bpe.txt", "w", encoding = "utf-8") as f:
    for line in corpus:
        f.write(line + "\n")

# Step 3: Initialize and train the ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(
    files = "temp_bpe.txt",
    vocab_size = 100,
    min_frequency = 1,
    special_tokens = ["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
)

# Step 4: Encode a new sentence
encoded = tokenizer.encode("Tokenization helps subword modeling.")
print("Tokens:", encoded.tokens)


Tokens: ['T', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n', 'Ġ', 'h', 'e', 'l', 'p', 's', 'Ġ', 's', 'u', 'b', 'w', 'o', 'r', 'd', 'Ġ', 'm', 'o', 'd', 'e', 'l', 'i', 'n', 'g', '.']
